In [2668]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function


import datetime
#imports

# Ideas

Raise error when len does not match before and after merging datasets (means that a player wasn't found -- likely because he is a junior or has two last names, etc.)

# To Do

Address last name issue

In [2669]:
batters_df = pd.read_csv('../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../data/individual_pitching_stats.csv')
misc_batters = pd.read_csv('../data/batters_teams.csv')
misc_pitchers = pd.read_csv('../data/pitchers_teams.csv')

In [2670]:
pitchers_df.loc[pitchers_df['last_name'] == 'Iglesias']

,last_name,first_name,player_id,year,p_bb_percent,batting_avg,xba,xslg,woba,xwoba,...,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 22
254,Iglesias,Raisel,628452,2022,0.0,0.107,0.134,0.218,0.142,0.15,...,0.248,0.176,0.22,-0.027,-0.004,-0.008,84.5,21.5,5.9,NaN


In [2671]:
batters_df[batters_df['last_name'].str.contains(" ")]

,last_name,first_name,player_id,year,b_bb_percent,batting_avg,xba,xslg,woba,xwoba,...,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 22
5,Cruz Jr.,Nelson,443558,2022,11.6,0.160,0.262,0.484,0.234,0.354,...,0.399,0.203,0.334,-0.102,-0.244,-0.120,89.5,8.6,10.2,NaN
18,Brantley Jr.,Michael,488726,2022,7.9,0.290,0.330,0.563,0.341,0.396,...,0.446,0.345,0.393,-0.040,-0.143,-0.055,89.2,11.8,8.5,NaN
60,Springer III,George,543807,2022,5.0,0.288,0.266,0.452,0.379,0.329,...,0.381,0.362,0.336,0.022,0.069,0.050,90.5,17.6,5.2,NaN
87,Voit III,Luke,572228,2022,18.5,0.143,0.150,0.271,0.250,0.280,...,0.287,0.231,0.244,-0.007,-0.104,-0.030,85.9,19.0,7.7,NaN
129,Bradley Jr.,Jackie,598265,2022,10.9,0.161,0.219,0.349,0.232,0.290,...,0.319,0.214,0.292,-0.058,-0.117,-0.058,89.0,8.5,2.3,NaN
207,Mancini III,Trey,641820,2022,7.5,0.236,0.317,0.562,0.280,0.388,...,0.500,0.315,0.430,-0.081,-0.229,-0.108,91.6,9.8,11.1,NaN
243,Mullins II,Cedric,656775,2022,8.2,0.205,0.257,0.489,0.307,0.363,...,0.405,0.268,0.335,-0.052,-0.133,-0.056,89.2,18.8,12.3,NaN
282,Guerrero Jr.,Vladimir,665489,2022,11.4,0.309,0.329,0.672,0.402,0.438,...,0.526,0.382,0.414,-0.020,-0.113,-0.036,91.5,9.4,14.5,NaN
285,Chisholm Jr.,Jazz,665862,2022,8.5,0.308,0.268,0.561,0.436,0.383,...,0.506,0.421,0.397,0.040,0.112,0.053,89.1,14.8,15.8,NaN
295,Gurriel Jr.,Lourdes,666971,2022,3.8,0.274,0.271,0.461,0.322,0.329,...,0.386,0.333,0.341,0.003,-0.036,-0.007,92.3,13.4,5.0,NaN


In [2672]:
len(batters_df)

335

In [2673]:
bat_xbadiff = batters_df[['last_name', 'first_name', 'xbadiff','batting_avg', 'xba']]
#Make a table with just the xBA difference

In [2674]:
pitch_xbadiff = pitchers_df[['last_name', 'first_name', 'xbadiff', 'batting_avg', 'xba']]
# pitch_xbadiff.sort_values('xba', ascending=False)

# Probable Pitchers

In [2675]:
url = 'https://www.mlb.com/probable-pitchers/2022-04-30'
#URL of where I will get the pitchers' names

In [2676]:
res = requests.get(url)
soup = BeautifulSoup(res.content)
#Making request to the site and storing it in soup

In [2677]:
pitchers = soup.find_all('a', {'class': 'probable-pitchers__pitcher-name-link'})
probable_pitchers = []
for i in pitchers:
    probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [2678]:
probable_pitchers

['<a class="probable-pitchers__pitcher-name-link" href="/player/merrill-kelly-518876">Merrill Kelly</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/miles-mikolas-571945">Miles Mikolas</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/luis-garcia-677651">Luis Garcia</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/jose-berrios-621244">Jose Berrios</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/joan-adon-672851">Joan Adon</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/logan-webb-657277">Logan Webb</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/jose-suarez-660761">Jose Suarez</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/vince-velasquez-592826">Vince Velasquez</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/shane-bieber-669456">Shane Bieber</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/cole

In [2679]:
probable_pitchers = [i.split('">')[1] for i in probable_pitchers] 
#Getting only the pitchers' names

In [2680]:
probable_pitchers = [i.split('</a')[0] for i in probable_pitchers] 
#Getting only the pitchers' names part 2

In [2681]:
probable_pitchers = [i.split() for i in probable_pitchers]
#Splitting name into first and last name columns

In [2682]:
df_probable_pitchers = pd.DataFrame(probable_pitchers)
#Setting list as Pandas DF 

In [2683]:
df_probable_pitchers = df_probable_pitchers.rename(columns={0: "first_name", 1: "last_name"})
#Renaming column

# Get Pitchers' Teams

In [2710]:
new_teams = soup.find_all('span', {'class':['probable-pitchers__team-name probable-pitchers__team-name--away', 'probable-pitchers__team-name probable-pitchers__team-name--home']})
teams_of_probable_pitchers = []
for i in new_teams:
    teams_of_probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [2711]:
teams_of_probable_pitchers

['<span class="probable-pitchers__team-name probable-pitchers__team-name--away">\n                                                D-backs\n                                            </span>',
 '<span class="probable-pitchers__team-name probable-pitchers__team-name--home">\n                                                Cardinals\n                                            </span>',
 '<span class="probable-pitchers__team-name probable-pitchers__team-name--away">\n                                                Astros\n                                            </span>',
 '<span class="probable-pitchers__team-name probable-pitchers__team-name--home">\n                                                Blue Jays\n                                            </span>',
 '<span class="probable-pitchers__team-name probable-pitchers__team-name--away">\n                                                Nationals\n                                            </span>',
 '<span class="probable-pitche

In [2714]:
teams_of_probable_pitchers = [x.strip(' ') for x in teams_of_probable_pitchers]
#Getting only the pitchers' names

In [2716]:
teams_of_probable_pitchers = [i.split('>\n') for i in teams_of_probable_pitchers] 
#Getting only the pitchers' names

In [2717]:
teams_of_probable_pitchers = teams_of_probable_pitchers = [x.strip('                                   ') for x in teams_of_probable_pitchers]


[['<span class="probable-pitchers__team-name probable-pitchers__team-name--away"',
  '                                                D-backs\n                                            </span>'],
 ['<span class="probable-pitchers__team-name probable-pitchers__team-name--home"',
  '                                                Cardinals\n                                            </span>'],
 ['<span class="probable-pitchers__team-name probable-pitchers__team-name--away"',
  '                                                Astros\n                                            </span>'],
 ['<span class="probable-pitchers__team-name probable-pitchers__team-name--home"',
  '                                                Blue Jays\n                                            </span>'],
 ['<span class="probable-pitchers__team-name probable-pitchers__team-name--away"',
  '                                                Nationals\n                                            </span>'],
 ['<s

In [2689]:
teams_of_probable_pitchers = [i.split('>\n')[1] for i in teams_of_probable_pitchers] 
#Getting only the pitchers' names

In [2690]:
#Getting only the pitchers' names part 2

In [2691]:
misc_pitchers = misc_pitchers[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [2692]:
misc_pitchers[['first_name', 'last_name']] = misc_pitchers['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [2693]:
misc_pitchers = misc_pitchers.drop(columns='Player')
#We don't need this full name column anymore

In [2694]:
misc_pitchers = misc_pitchers[['last_name', 'first_name', 'Team']]
#reordering columns

# Get Batters' Teams

In [2695]:
misc_batters = misc_batters[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [2696]:
misc_batters[['first_name', 'last_name']] = misc_batters['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [2697]:
misc_batters = misc_batters.drop(columns='Player')
#We don't need this full name column anymore

In [2698]:
misc_batters = misc_batters[['last_name', 'first_name', 'Team']]
#Reordering columns

In [2699]:
len(df_probable_pitchers)

30

In [2700]:
pitchers_df['first_name'] = pitchers_df['first_name'].str.strip()
batters_df['first_name'] = batters_df['first_name'].str.strip()
#Removing empty space in these column entries for example this makes ' Cabrera' => 'Cabrera'

In [2701]:
df_probable_pitchers

,first_name,last_name
0,Merrill,Kelly
1,Miles,Mikolas
2,Luis,Garcia
3,Jose,Berrios
4,Joan,Adon
5,Logan,Webb
6,Jose,Suarez
7,Vince,Velasquez
8,Shane,Bieber
9,Cole,Irvin


batters_df and pitchers_df front names all have a space in front of them and that's why merge is not working

# Attaching according Team Name to each player in all DFs

The datasets on Baseball Savant do not include the team name with they players, so I will add them in

In [2702]:
df_probable_pitchers = pd.merge(df_probable_pitchers, misc_pitchers, on=['last_name', 'first_name'], how='left')
pitchers_df = pd.merge(pitchers_df, misc_pitchers, on=['last_name', 'first_name'], how='inner')
batters_df = pd.merge(batters_df, misc_batters, on=['last_name', 'first_name'], how='inner')
#Adding team to all of the players
#Joining probables as a left join so it will include all of the probables without exception -- may make one of their 

In [2703]:
len(df_probable_pitchers)

31

In [2704]:
df_probable_pitchers = pd.merge(df_probable_pitchers, pitchers_df, on=['last_name', 'first_name', 'Team'], how='left')

# Which player to pick for BTS

In [2705]:
pp_xba = df_probable_pitchers[['last_name', 'first_name', 'Team', 'batting_avg', 'xba', 'p_bb_percent']]
#How can I compare this to batters df??

In [2706]:
pp_xba.rename(columns= {'xba': 'pitcher_xba'})
#Eventually might move this to another df or merge, unclear

,last_name,first_name,Team,batting_avg,pitcher_xba,p_bb_percent
0,Kelly,Merrill,ARI,0.244,0.259,7.9
1,Mikolas,Miles,STL,0.205,0.224,4.4
2,Garcia,Luis,HOU,0.182,0.309,0.0
3,Garcia,Luis,HOU,0.196,0.204,6.6
4,Garcia,Luis,SD,0.182,0.309,0.0
5,Garcia,Luis,SD,0.196,0.204,6.6
6,Berrios,Jose,TOR,0.289,0.313,8.2
7,Adon,Joan,WAS,0.282,0.320,11.1
8,Webb,Logan,SF,0.234,0.258,5.1
9,Suarez,Jose,LAA,0.224,0.302,15.5


One thing I could do is... switch the one column with the next, and then merge it with the batter's df
So for example if Oakland is playing Cleveland, I can switch the pitchers with each other. So Oakland's pitcher will be on Cleveland, and then I will have the pitcher xba to match with the team. Then I can basically set that for every member of the team, and then add that with the hitter xba and see if it meets the criteria

In [2707]:
pp_xba.sort_values(by='xba', ascending=False)

,last_name,first_name,Team,batting_avg,xba,p_bb_percent
26,Hernandez,Carlos,KC,0.333,0.326,7.7
7,Adon,Joan,WAS,0.282,0.320,11.1
6,Berrios,Jose,TOR,0.289,0.313,8.2
10,Velasquez,Vince,CWS,0.288,0.312,11.9
2,Garcia,Luis,HOU,0.182,0.309,0.0
4,Garcia,Luis,SD,0.182,0.309,0.0
9,Suarez,Jose,LAA,0.224,0.302,15.5
12,Irvin,Cole,OAK,0.226,0.296,5.6
31,Brieske,Beau,DET,0.167,0.295,10.0
22,Dunning,Dane,TEX,0.293,0.295,9.5


In [2708]:
batters_df.loc[batters_df['Team'] == 'BAL']

,last_name,first_name,player_id,year,b_bb_percent,batting_avg,xba,xslg,woba,xwoba,...,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 22,Team
9,Chirinos,Robinson,455139,2022,13.6,0.189,0.112,0.177,0.273,0.214,...,0.280,0.166,0.077,0.066,0.059,84.1,16.8,4.0,NaN,BAL
116,Odor,Rougned,596059,2022,7.7,0.200,0.182,0.260,0.268,0.256,...,0.300,0.285,0.018,0.029,0.012,87.5,19.2,0.0,NaN,BAL
125,Urias,Ramon,602104,2022,6.3,0.203,0.299,0.449,0.229,0.342,...,0.273,0.400,-0.096,-0.212,-0.113,91.7,8.3,4.5,NaN,BAL
164,Bemboom,Anthony,621532,2022,11.1,0.130,0.232,0.278,0.200,0.278,...,0.176,0.319,-0.102,-0.104,-0.078,87.0,3.7,0.0,NaN,BAL
172,Mateo,Jorge,622761,2022,7.2,0.238,0.266,0.385,0.274,0.308,...,0.366,0.409,-0.028,-0.083,-0.034,88.4,16.6,4.8,NaN,BAL
175,Santander,Anthony,623993,2022,17.3,0.222,0.253,0.431,0.365,0.377,...,0.304,0.349,-0.031,-0.034,-0.012,89.3,16.0,6.5,NaN,BAL
212,Gutierrez,Kelvin,642721,2022,12.5,0.148,0.225,0.395,0.219,0.319,...,0.211,0.326,-0.077,-0.210,-0.100,90.0,4.3,5.3,NaN,BAL
253,Mountcastle,Ryan,663624,2022,2.8,0.232,0.300,0.444,0.253,0.332,...,0.327,0.427,-0.068,-0.154,-0.079,88.7,15.6,8.2,NaN,BAL
299,Hays,Austin,669720,2022,10.4,0.279,0.265,0.420,0.374,0.337,...,0.358,0.340,0.014,0.051,0.037,88.8,11.0,3.8,NaN,BAL


In [2709]:
# batters_df.loc[batters_df['last_name'] == 'Guererro']
# To fix later

# Which Hitters to pick for BTS

Now we need to see 